In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.conf import SparkConf
import pyspark.sql.functions as func

import pandas as pd

In [ ]:
# create an spark session
spark_conf = SparkConf().setAppName('my_etl')
spark_conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:2.7.3,org.postgresql:postgresql:9.4.1211')
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()

sc = spark.sparkContext